In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#import pymongo
import datetime

from sqlalchemy import create_engine
import json
from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)

In [2]:
weather_df =pd.read_csv('Resources/US_WeatherEvents_2016-2019.csv')

FileNotFoundError: [Errno 2] File Resources/US_WeatherEvents_2016-2019.csv does not exist: 'Resources/US_WeatherEvents_2016-2019.csv'

In [ ]:
weather_df.head()

In [ ]:
filt = (weather_df["StartTime(UTC)"].str.contains('2018-01-01'))
selected_df = weather_df.loc[filt]

In [ ]:
selected_df

In [ ]:
selected_df2 = selected_df[['Type','Severity','LocationLat','LocationLng']]

In [ ]:
selected_df2.to_csv('Clean/weather_cleaned.csv',index=False)

In [ ]:
pd.read_csv('Clean/weather_cleaned.csv')

In [ ]:
selected_df2['Type'].unique()

In [ ]:
filt = (weather_df["StartTime(UTC)"].str.contains('2018-01-01'))&(weather_df['Severity']=='Severe')
severe_df = weather_df.loc[filt]

In [ ]:
severe_df.columns

In [ ]:
severe_df2 = severe_df[['Type','LocationLat','LocationLng']]

In [ ]:
severe_df2['Type'].unique()

In [ ]:
severe_df3=severe_df2.dropna()

In [ ]:
severe_df3['Type'].unique()

In [ ]:
severe_df3.to_csv('Clean/severe_weather_cleaned2.csv',index=False)

In [ ]:
severe_df3 = pd.read_csv('Clean/severe_weather_cleaned2.csv')

In [ ]:
severe_df3

## Airport Dataclean

In [ ]:
ori_gp = df.groupby(['ORIGIN'])

In [ ]:
delay_ds = ori_gp['DEP_DELAY'].mean()
delay_ds
delay_df= pd.DataFrame(delay_ds)
delay_df

In [ ]:
flight_ds=ori_gp['FL_DATE'].count()
flight_ds
flight_df= pd.DataFrame(flight_ds)

In [ ]:
cancel_ds=ori_gp['CANCELLED'].sum()
cancel_ds
cancel_df= pd.DataFrame(cancel_ds)

In [ ]:
airport_df = pd.read_csv('Resources/completed_airports.csv')

In [ ]:
airport_df.head()
airport_df = airport_df[['IATA','Airport','City','STATE','LATITUDE','LONGITUDE']]

In [ ]:
merged_df = pd.merge(flight_df,delay_df,left_index=True, right_index=True, how='inner')

In [ ]:
merged_df = pd.merge(merged_df,cancel_df,left_index=True, right_index=True, how='inner')

In [ ]:
merged_df.reset_index(inplace = True)

In [ ]:
merged_df

In [ ]:
air_df= pd.merge(merged_df,airport_df,left_on='ORIGIN',right_on='IATA',how='left')

In [ ]:
air_df = air_df.dropna(how='any')

In [ ]:
air_df = air_df.drop(columns=['IATA'])
air_df

In [ ]:
air_df.columns=['IATA_code','Flights','AvgDelay','Cancelled','Fullname','City','State','Lat','Lon']

In [ ]:
air_df

In [ ]:
air_df.to_csv('Clean/airport_cleaned.csv',index=False)

In [ ]:
air_df = pd.read_csv('Clean/airport_cleaned.csv')

In [ ]:
air_na_df=air_df.fillna('null')

In [ ]:
air_na_df.to_csv('Clean/airport_na_cleaned.csv',index=False)

In [ ]:
pd.read_csv('Clean/airport_na_cleaned.csv')

# Flight cleanup

In [ ]:
flight_df = df.copy()
flight_df

In [ ]:
flight_df['DEP_TIME'] = flight_df['DEP_TIME'].loc[~flight_df['DEP_TIME'].isna()].astype(int).astype(str)
flight_df['DEP_TIME']

In [ ]:
flight_df['DEP_TIME'].loc[flight_df['DEP_TIME'].str.len()==3] = '0'+flight_df['DEP_TIME'].loc[flight_df['DEP_TIME'].str.len()==3]


In [ ]:
flight_df['DEP_TIME'] = flight_df['DEP_TIME'].str[0:2]+':'+flight_df['DEP_TIME'].str[2:4]

In [ ]:
flight_df['DEP_TIME']

In [ ]:
flight_df=flight_df[['FL_DATE','OP_CARRIER','OP_CARRIER_FL_NUM','ORIGIN', 'DEST','AIR_TIME','DISTANCE','DEP_TIME', 'DEP_DELAY','CANCELLED']]

In [ ]:
flight_df

In [ ]:
flight_df.columns=['Date','Airline','Flight_Num','Origin','Destination','Air_Time','Distance','Dep_Time','Delay', 'Cancelled']

In [ ]:
flight_df

In [ ]:
flight_df['Cancelled']=flight_df['Cancelled'].replace(0, 'No')

In [ ]:
flight_df['Cancelled']=flight_df['Cancelled'].replace(1, 'Yes')

In [ ]:
flight_df.to_csv('Clean/flights_cleaned.csv',index=False)

In [ ]:
pd.read_csv('Clean/flights_cleaned.csv')

In [ ]:
flight_df

In [ ]:
## Airline info

In [ ]:
airline_df = pd.read_csv('Resources/completed_airlines.csv')
airline_df

In [ ]:
flight_extra_df= pd.merge(flight_df,airline_df,left_on='Airline',right_on='IATA', how='left')
flight_extra_df

In [ ]:
flight_extra_df.drop(columns=['IATA'],inplace= True)

In [ ]:
flight_extra_df

In [ ]:
flight_extra_df.columns=['Date','Airline','Flight_Num','Origin','Destination','Air_Time','Distance','Dep_Time','Delay', 'Cancelled', 'Airline_Info']

In [ ]:
flight_extra_df.to_csv('Clean/flights__extra_cleaned.csv',index=False)

In [ ]:
flight_extra_df = pd.read_csv('Clean/flights__extra_cleaned.csv')

In [ ]:
flight_extra_na_df=flight_extra_df.fillna('null')

In [ ]:
flight_extra_na_df.to_csv('Clean/flights_extra_na_cleaned.csv',index=False)

In [ ]:
flight_extra_na_df = pd.read_csv('Clean/flights_extra_na_cleaned.csv')
flight_extra_na_df

In [ ]:
flight_extra_na_df['Airline'].unique()

### mongo database

In [ ]:
# database: air_df, flight_extra_df

In [ ]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')

In [ ]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
db = client.project2_db

In [ ]:
db.airport.insert_many(air_df.to_dict('records'))

In [ ]:
db.flight.insert_many(flight_extra_df.to_dict('records'))

In [ ]:
listings = db.airport.find()
airport_data=[]
for listing in listings:
    airport_data.append(listing)
print(airport_data[1])

In [ ]:
db.flight

In [ ]:
# database without NA: air_na_df, flight_extra_na_df

In [ ]:
db.flight_na.insert_many(flight_extra_na_df.to_dict('records'))

In [ ]:
db.airport_na.insert_many(air_na_df.to_dict('records'))

In [3]:
db.severe_180101_na.insert_many(severe_df3.to_dict('records'))

NameError: name 'db' is not defined

In [ ]:
selected_df2